In [1]:
pip install requests beautifulsoup4


  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached soupsieve-2.5-py3-none-any.whl (36 kB)


You should consider upgrading via the 'c:\Users\ASUS\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install requests

You should consider upgrading via the 'c:\Users\ASUS\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


# Ekstraksi Data Ulasan

In [3]:
import requests
from bs4 import BeautifulSoup

In [28]:
url = 'https://shopee.co.id/Sandal-Wanita-Flat-Shoes-Heels-Chunky-SYDTRA-Vega-Sendal-Tali-Kekinian-Kasual-i.961807342.24861624163?publish_id=&sp_atk=0f2b6aee-97be-400c-8fc2-e7a07193943d&xptdk=0f2b6aee-97be-400c-8fc2-e7a07193943d'
response = requests.get(url)

In [47]:
soup = BeautifulSoup(response.text, 'html.parser')

In [56]:
reviews = soup.find_all('div', class_= 'shopee-product-rating__main')

In [61]:
data = []
for review in reviews:
    rating = review.find('div', class_='shopee-product-rating__rating').text
    username = review.find('a', class_='shopee-product-rating__author-name').text
    review_text = review.find('div', style='position: relative; box-sizing: border-box; margin: 15px 0px; font-size: 14px; line-height: 20px; color: rgba(0, 0, 0, 0.87); word-break: break-word; white-space: pre-wrap;').text
    review_date = review.find('div', class_='shopee-product-rating__time').text
    data.append([rating, username, review_text, review_date])


In [58]:
import pandas as pd

In [59]:
df = pd.DataFrame(data, columns=['rating', 'username', 'review_text', 'review_date'])
df

,rating,username,review_text,review_date


# Penanganan Pagination

In [60]:
next_page = soup.find('a', class_='next-page')
while next_page:
    next_page_url = next_page['herf']
    response = requests.get(next_page_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    reviews = soup.find_all('div', class_='css-1k41fl7')
    
    for review in reviews:
        rating = review.find('div', class_='shopee-product-rating__rating').text
        username = review.find('a', class_='shopee-product-rating__author-name').text
        review_text = review.find('div', style='position: relative; box-sizing: border-box; margin: 15px 0px; font-size: 14px; line-height: 20px; color: rgba(0, 0, 0, 0.87); word-break: break-word; white-space: pre-wrap;').text
        review_date = review.find('div', class_='shopee-product-rating__time').text
        
    
    next_page = soup.find('a', class_='next-page')

# Pembersihan dan Penyimpanan Data

In [55]:
import csv

with open('reviews.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Rating', 'Review Text', 'Review Date', 'Username']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for review in reviews:
        rating = review.find('div', class_='shopee-product-rating__rating').text
        username = review.find('a', class_='shopee-product-rating__author-name').text
        review_text = review.find('div', style='position: relative; box-sizing: border-box; margin: 15px 0px; font-size: 14px; line-height: 20px; color: rgba(0, 0, 0, 0.87); word-break: break-word; white-space: pre-wrap;').text
        review_date = review.find('div', class_='shopee-product-rating__time').text
        
        writer.writerow({'Rating': rating, 'Review Text': review_text, 'Review Date': review_date, 'Username': username})